# Weekly speed averages

In [ ]:
import altair as alt
import pandas as pd

from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis import styleguide
from segment_speed_utils.project_vars import SEGMENT_GCS

In [ ]:
df = pd.read_parquet(
    f"{SEGMENT_GCS}rollup/speeds_by_peak_daytype_apr2023.parquet"
)

In [ ]:
test = df[(df.schedule_gtfs_dataset_key=="015d67d5b75b5cf2b710bbadadfb75f5") & 
  (df.shape_id=="102") 
  #(df.stop_sequence==20)
         ].reset_index(drop=True)

In [ ]:
test = test.assign(
    category = test.apply(
        lambda x: x.weekday_weekend + " " + x.peak_offpeak, axis=1)
)

In [ ]:
chart = (alt.Chart(test)
         .mark_bar()
         .encode(
             x=alt.X("category:O"),
             y=alt.Y("p50_mph:Q"),
             column=alt.Column("stop_sequence:O"),
             color=alt.Color("category:N")
         )
        )

chart

In [ ]:
(alt.Chart(test)
         .mark_line()
         .encode(
             x=alt.X("stop_sequence:O"),
             y=alt.Y("p50_mph:Q"),
             color=alt.Color("category:N")
         )
        )

In [ ]:
df[df.p20_mph > 3].groupby(["schedule_gtfs_dataset_key", "shape_id", 
            "stop_sequence", "stop_id"]
          ).agg({"peak_offpeak": "count"}
).reset_index().query('peak_offpeak == 4')

In [ ]:
df[(df.schedule_gtfs_dataset_key=="015d67d5b75b5cf2b710bbadadfb75f5") & 
  (df.shape_id=="102") & 
  (df.stop_sequence==20)]

## Further route-direction aggregation

Since we're not sure if `stop_sequence-stop_id` line up across shapes...we should add a column that tracks `prior_stop_id-stop_id` and use that to aggregate. The segments that do line up (between the same stops) would get aggregated, the ones that don't line up nicely would remain (with much smaller `n_trips`).